In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np

2023-07-12 14:27:49.548394: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-12 14:27:49.576372: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-12 14:27:50.004573: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
import tensorflow_datasets as tfds

dataset = tfds.load(
          'imagenet2012',
          data_dir = '/home/ezraf/dataset/ilsvrc2012',
          download = True
          )
val = dataset['validation']

/home/ezraf/miniconda3/envs/tf/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-07-12 14:27:50.732992: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-07-12 14:27:50.751382: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-07-12 14:27:50.751501: I tensorflow/compiler/xla/s

In [3]:
import PIL.Image as Image
IMAGE_SHAPE = (224, 224)
grace_hopper = tf.keras.utils.get_file('image.jpg','https://storage.googleapis.com/download.tensorflow.org/example_images/grace_hopper.jpg')
grace_hopper = Image.open(grace_hopper).resize(IMAGE_SHAPE)

In [4]:
grace_hopper = (np.array(grace_hopper))/255.0
grace_hopper.shape

(224, 224, 3)

In [5]:
model = tf.keras.Sequential([hub.KerasLayer("https://tfhub.dev/sayakpaul/vit_s16_classification/1")])

In [6]:
result = model.predict(grace_hopper[np.newaxis, ...])
result.shape

2023-07-12 14:27:52.555949: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,224,224,3]
	 [[{{node inputs}}]]
2023-07-12 14:27:52.610905: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder' with dtype float and shape [?,224,224,3]
	 [[{{node Placeholder}}]]
2023-07-12 14:27:52.972142: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x7f5bd002c360 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-07-12 14:27:52.972172: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): NVIDIA GeForce RTX 309

1/1 [==============================] - 4s 4s/step


2023-07-12 14:27:56.533165: I ./tensorflow/compiler/jit/device_compiler.h:180] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


(1, 1000)

In [11]:
predicted_class = tf.math.argmax(result[0], axis=-1)
predicted_class += 1

In [12]:
labels_path = tf.keras.utils.get_file('ImageNetLabels.txt','https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt')
imagenet_labels = np.array(open(labels_path).read().splitlines())

In [13]:
predicted_class_name = imagenet_labels[predicted_class]
predicted_class_name

'military uniform'